In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\abooth\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [3]:
# this will be appended Mongo
scraped_data = {}

####################################################################

# First, get NASA News

url = "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(1)

soup = BeautifulSoup(browser.html)
slide = soup.find("li", {"class": "slide"})
news_title = slide.find("div", {"class": "content_title"}).text.strip()
news_p = slide.find("div", {"class": "article_teaser_body"}).text.strip()

########################################################################

# NEXT, Get FEATURED URL

base = "https://www.jpl.nasa.gov"
url = f"{base}/spaceimages/?search=&category=Mars"
browser.visit(url)
time.sleep(1)

browser.find_by_id("full_image").click()
time.sleep(1)

browser.find_link_by_partial_text("more info").click()
time.sleep(1)

soup = BeautifulSoup(browser.html)
image = soup.find("img", {"class": "main_image"})

featured_image_url  = base + image["src"]

########################################################################

# MARS FACTS
url = "https://space-facts.com/mars/"
browser.visit(url)
time.sleep(1)

dfs = pd.read_html(browser.html)
df = dfs[0]
df.columns = ["Statistic", "Value"]
mars_facts = df.to_html(index=False)

########################################################################

# HEMISPHERE Data

base = "https://astrogeology.usgs.gov"
url = f"{base}/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
time.sleep(1)

soup = BeautifulSoup(browser.html)
links = soup.find("div", {"class": "results"}).findAll("a", {"class": "itemLink"})

#filter out non image results
realLinks = []
for link in links:
    image = link.find("img")
    if (image):
        realLinks.append(base + link["href"]) # append the base url

# LOOP through each image link, click, grab the image info
hemisphere_data = []
for realLink in realLinks:
    browser.visit(realLink)
    time.sleep(1)
    
    soup = BeautifulSoup(browser.html)
    hemi_url = soup.find("ul").find("li").find("a")["href"]
    hemi_title = soup.find("h2", {'class', "title"}).text.split(" Enhanced")[0]
    
    hemisphere_data.append({"title": hemi_title, "url": hemi_url})

#exit browser
browser.quit()

#append data
scraped_data["news_title"] = news_title
scraped_data["news_p"] = news_p
scraped_data["featured_image_url"] = featured_image_url
scraped_data["mars_facts"] = mars_facts
scraped_data["hemispheres"] = hemisphere_data

In [4]:
scraped_data

{'news_title': "A Martian Roundtrip: NASA's Perseverance Rover Sample Tubes",
 'news_p': "Marvels of engineering, the rover's sample tubes must be tough enough to safely bring Red Planet samples on the long journey back to Earth in immaculate condition.",
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16567_hires.jpg',
 'mars_facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Statistic</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period: